## 安然数据
首先确定目的是为了进行数据分析，因此需要从数据分析的过程去看看怎么样使用 R 来进行相关过程。本次将以安然的部分数据来进行分析，重点将以 EDA 的角度来进行：

1. 读取数据
2. 数据清理
3. **探索分析**
    * 单变量分析
    * 双变量分析
    * 多变量分析
4. 建模

In [3]:
library(ggplot2)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### 读取数据
R 同样可以读取多种类型的数据文件，例如Excel、XML、SPSS、SAS 和 MATLAB，剪贴板，甚至是网络数据。常用的函数中需要注意，`read.csv()` 和 `read.csv2()` 的差异——后者默认是使用`;` 作为分隔符，`,` 作为千分位的 CSV 文件类型。对于其他异常数据作为缺失值，可以通过 `na.strings` 参数进行调整——`na.strings=c("NULL", "#N/A", "#DIV/0")`。

<font size=1 color="green">补充信息：
    <ol>
        <li>如果需要保存相应的数据，那么可以使用 <code>write.table</code> 等函数</li>
        <li>此外如果需要<b>交互</b>，可以使用 <code>readline()</code></li>
        <li>下载数据，可以使用 <code>download.file()</code> 或者 <code>RCurl</code>包获取 HTML 或 XML 数据，而 <code>XML</code> 包中的相关函数可以进行页面解析
    </ol>
</font>

In [4]:
df <- read.csv("./data/enron.csv")

### 清理数据
通过相应的函数获取到了数据信息，那么接下来需要了解数据信息以确定接下来的数据清洗的相对应的工作。Python 中 Pandas 提供了很多方便的方式进行数据访问，同样地在 R 中可以使用一些简单的函数来完成相应的工作：

1. 数据总体信息，`str()`
2. 数据维度信息，`dim()`，此外还可以使用 `NROW()` 和 `NCOL()` 来对非

In [5]:
head(df)

Name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,⋯,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,⋯,304805,152,FALSE,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,⋯,NaN,NaN,FALSE,NaN,NaN,NaN,NaN,NaN,182466,257817
BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,29,39,⋯,NaN,864523,FALSE,1757552,-560222,477,465,566,916197,5243487
BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,⋯,1586055,2660303,FALSE,3942714,NaN,267102,NaN,NaN,5634343,10623258
BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,NaN,NaN,⋯,NaN,69,FALSE,145796,-82782,239671,NaN,NaN,827696,63014
BAZELIDES PHILIP J,NaN,684694,NaN,NaN,NaN,1599641,NaN,NaN,NaN,⋯,93750,874,FALSE,NaN,NaN,80818,NaN,NaN,860136,1599641


In [23]:
read.table(pipe("pbpaste"), sep="::")

ERROR: Error in scan(file, what = "", sep = sep, quote = quote, nlines = 1, quiet = TRUE, : 'sep' 值不正確: 大小只能為一個位元組


In [30]:
scan("./data/R_Paste.docx")

ERROR: Error in scan("./data/R_Paste.docx"): scan() 需要 'a real'，而不是 'PK'
